In [ ]:
pip install selenium

In [ ]:
#Import packages
import pandas as pd
from selenium import webdriver
from time import sleep
import sys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
import pickle

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#Chrome Driver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
url_set = set()

In [ ]:
#Crawls through MCF searching for job_title, keep only the links to the jobs
#Limit set to fresh and entry level positions
def search_mcf(job_title):
  for i in range(9999999999):
    try:
      title=[]
      driver.get(f'https://www.mycareersfuture.gov.sg/search?search={job_title}&positionLevel=Fresh%2Fentry%20level&positionLevel=Junior%20Executive&sortBy=relevancy&page={i}')
      assert 'MyCareersFuture' in driver.title
      title_elem = "//a[@data-testid='job-card-link']"
      element = WebDriverWait(driver, 10).until(
              EC.presence_of_element_located((By.XPATH,title_elem))
          )
      title = driver.find_elements(By.XPATH,title_elem)
      if not title:
        break
      for i in title:
        url_set.add(i.get_attribute('href'))
    except Exception as e:
      print(e)
      break

In [ ]:
job_titles = ["Data",
"Tech", 
"Software",
"Product Manager",
"Analyst",
"Analytics",
"Data Scientist",
"Data Science",
"AI",
"ML",
"NLP",
"Quant",
"Data Engineer",
"Artificial Intelligence",
"Machine Learning", 
"Quantitative",
"Business Intelligence",
"Computer Vision",
"Natural Language Processing",
"Data Pipelines"
]

In [ ]:
for job_title in job_titles:
  print(job_title)
  search_mcf(job_title)

In [ ]:
len(url_set)

1531

In [ ]:
#Saved in /content
with open('drive/MyDrive/url_set.pkl', 'wb') as f:
    pickle.dump(url_set, f)

In [ ]:
with (open('drive/MyDrive/url_set.pkl', "rb")) as openfile:
  url_set = pickle.load(openfile)

In [ ]:
len(url_set)

In [ ]:
result_dict = {'url':[],'job_title':[],'company':[],'code':[], 'address':[], 'emp_type':[], 'level':[],'sector':[],'pay_range':[],'description':[]}

In [ ]:
count = 0
for i in url_set:
  count+=1
  print(count)
  try: 
    driver.get(i)
    element = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.ID,"description-content")))
  except Exception:
    print(Exception)
    continue
  try:          
    result_dict['job_title'].append(driver.find_element(By.ID,"job_title").get_attribute('innerHTML'))
  except:
    result_dict['job_title'].append(None)
  try:
    result_dict['company'].append(driver.find_element(By.XPATH,"//p[@data-testid='company-hire-info']").get_attribute('innerHTML'))
  except:
    result_dict['company'].append(None)
  try:
    result_dict['code'].append(driver.find_element(By.XPATH,"//span[@data-cy='jobinfo__jobpostid--span']").get_attribute('innerHTML'))
  except:
    result_dict['code'].append(None)
  try:
    result_dict['address'].append(driver.find_element(By.ID,"scroll_to_location_map").get_attribute('innerHTML'))
  except:
    result_dict['address'].append(None)
  try:
    result_dict['emp_type'].append( driver.find_element(By.ID,"employment_type").get_attribute('innerHTML'))
  except:
    result_dict['emp_type'].append(None)
  try:
    result_dict['level'].append(driver.find_element(By.ID,"seniority").get_attribute('innerHTML'))
  except:
    result_dict['level'].append(None)
  try:
    result_dict['sector'].append(driver.find_element(By.ID,"job-categories").get_attribute('innerHTML'))
  except:
    result_dict['sector'].append(None)
  try:
    result_dict['pay_range'].append(driver.find_element(By.XPATH,"//span[@data-cy='salary-range']").get_attribute('innerHTML'))
  except:
    result_dict['pay_range'].append(None)
  try:
    result_dict['description'].append(driver.find_element(By.ID,"description-content").get_attribute('innerHTML'))
  except:
    result_dict['description'].append(None)
  result_dict['url'].append(i)


In [ ]:
mcf_jobs = pd.DataFrame(result_dict)

In [ ]:
mcf_jobs.to_csv('drive/MyDrive/mcf_jobs.csv')

In [ ]:
df = pd.read_csv('drive/MyDrive/mcf_jobs.csv')

In [ ]:
df.head()

In [ ]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [ ]:
import html.parser
html_parser = html.parser.HTMLParser()


In [ ]:
result = []
import html
for i in df['description']:
  i = cleanhtml(i)
  i = html.escape(i)
  result.append(i)
df['description-clean'] = result

In [ ]:
df.head(6)

,Unnamed: 0,url,job_title,company,code,address,emp_type,level,sector,pay_range,description,description-clean
0,0,https://www.mycareersfuture.gov.sg/job/custome...,PRODUCTION CONTROL MANAGER,SNL LOGISTICS PTE LTD,MCF-2022-0741372,31 GUL CIRCLE 629569,"Permanent, Full Time",Junior Executive,"Customer Service, General Management, Manufact...","<div class=""lh-solid""><span class=""dib"">$2,000...",<p><strong>JOB DESCIPTION</strong></p>\n<ul>\n...,JOB DESCIPTION\n\n planning and organising pr...
1,1,https://www.mycareersfuture.gov.sg/job/enginee...,Design Engineer ( Mechanical / Electrical),JAMCO AERO DESIGN &amp; ENGINEERING PRIVATE LI...,MCF-2022-0668683,NaN,Full Time,Junior Executive,Engineering,"<div class=""lh-solid""><span class=""dib"">$2,500...",<p><strong>SUMMARY</strong></p>\n<ul>\n <li>T...,SUMMARY\n\n This position is responsible...
2,2,https://www.mycareersfuture.gov.sg/job/sales/b...,Business Development Executive,AIRPAK EXPRESS PTE LTD,MCF-2022-0694935,"TECHPLAS INDUSTRIAL BUILDING, 45 CHANGI SOUTH ...","Part Time, Permanent",Fresh/entry level,Sales / Retail,"<div class=""lh-solid""><span class=""dib"">$3,200...",<p><strong>Job description</strong></p>\n<p>Wh...,Job description\nWho we are:\nWe are a logisti...
3,3,https://www.mycareersfuture.gov.sg/job/banking...,Senior / Data Scientist,SINGAPORE EXCHANGE LIMITED,MCF-2022-0682524,"SGX CENTRE I, 2 SHENTON WAY 068804","Permanent, Full Time",Professional,"Banking and Finance, Information Technology, R...","<div class=""lh-solid""><span class=""dib"">$9,000...",<p>The ideal candidate should have a good unde...,The ideal candidate should have a good underst...
4,4,https://www.mycareersfuture.gov.sg/job/archite...,8890-Sales Consultant [ Digital Software| Saas...,THE SUPREME HR ADVISORY PTE. LTD.,MCF-2022-0572683,"SHENTON HOUSE, 3 SHENTON WAY 068805","Permanent, Full Time",Junior Executive,"Architecture / Interior Design, Information Te...","<div class=""lh-solid""><span class=""dib"">$3,000...",<p><strong>Sales Consultant (Digital Software)...,Sales Consultant (Digital Software)\nLocation:...
5,5,https://www.mycareersfuture.gov.sg/job/logisti...,Logistics Analyst ►Degree / Logistics MNC #526,MTC CONSULTING PTE. LTD.,MCF-2022-0745250,NaN,"Permanent, Full Time",Fresh/entry level,Logistics / Supply Chain,"<div class=""lh-solid""><span class=""dib"">$3,600...",<p>➤ Basic <strong>$3400 - $4400</strong></p>\...,➤ Basic $3400 - $4400\n➤ AWS + Bonus (Full Ben...


In [ ]:
df.to_csv('drive/MyDrive/mcf_processed.csv')